In [2]:
# Tutorial 101_FileIO.py

import sys, os
# Add the igl library to the modules search path
sys.path.insert(0, os.getcwd() + "/../")
import igl

debug = False

# Load a mesh in OFF format
v, f, _ = igl.readOFF(igl.tutorial_path + "cube.off")#, read_normals=False)

# Print the vertices and faces matrices 
if debug:
    print("Vertices: \n", v, sep='')
    print("Faces: \n", f, sep='')

# Save the mesh in OBJ format
ret = igl.writeOBJ(igl.tutorial_path + "cube.obj", v, f)
if debug:
    print(ret)

In [3]:
# Tutorial 102_DrawMesh.py

import sys, os
# Add the igl library to the modules search path
sys.path.insert(0, os.getcwd() + "/../")
import igl

# Load a mesh in OFF format
v, f, _ = igl.readOFF(igl.tutorial_path + "bunny.off")#, read_normals=False)

# Plot the mesh
viewer = igl.Viewer()
viewer.set_mesh(v, f)
viewer.launch()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [6]:
# Tutorial 103_Events.py

import sys, os
import ipywidgets as widgets
# Add the igl library to the modules search path
sys.path.insert(0, os.getcwd() + "/../")
import igl

def select(change):
    if change["new"] == 1:
        #viewer.data().clear()
        viewer.set_mesh(v1, f1)
        #viewer.core.align_camera_center(V1,F1)
    if change["new"] == 2:
        viewer.set_mesh(v2, f2)

#  Load two meshes
v1, f1, _ = igl.readOFF(igl.tutorial_path + "bumpy.off")#, read_normals=False)
v2, f2, _ = igl.readOFF(igl.tutorial_path + "fertility.off")#,read_normals=False)

viewer = igl.Viewer()

# Create a widget and assign callback to switch between the two loaded meshes
viewer.add_widget(widgets.Dropdown(
    options={'Bump Mesh': 1, 'Fertility Mesh': 2},
    value=1,
    description='Select Mesh',
), select)

viewer.set_mesh(v1, f1)
viewer.launch()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Dropdown(description='Select Mesh', options={'Bump Mesh': 1, 'Fertility Mesh': 2}, value=1)

In [12]:
# Tutorial 104_Colors.py

import sys, os
import ipywidgets as widgets
# Add the igl library to the modules search path
sys.path.insert(0, os.getcwd() + "/../")
import igl
import matplotlib.pyplot as plt
import numpy as np

# Load a mesh in OFF format
v, f, _ = igl.readOFF(igl.tutorial_path + "screwdriver.off")

# Use the z coordinate as a scalar field over the surface
def get_colors(inp, colormap, vmin=None, vmax=None):
    norm = plt.Normalize(vmin, vmax)
    return colormap(norm(inp))

# Compute per-vertex colors
z = v[:, 2]
#c = get_colors(z, plt.cm.jet, vmin=np.min(z), vmax=np.max(z))
c = igl.jet(z, True)

# Plot the mesh
viewer = Vieweri()
viewer.set_mesh(v, f, c)

# Add per-vertex colors
# viewer.data().set_colors(C)

# Launch the viewer
viewer.launch()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [10]:
from pythreejs import *
from IPython.display import display

class Vieweri():
    def __init__(self):
        #print("Viewer initialization")
        self.data = {}
        self.cam = PerspectiveCamera(position=[0, 0, 1], lookAt=[0, 0, 0],fov=30,
                        children=[DirectionalLight(color='white', position=[0, 0, 1], intensity=0.6)])
        self.renderer = Renderer(camera=self.cam, scene = Scene(), controls=[OrbitControls(controlling=self.cam)], 
                    width=600, height=600, antialias=False)
        self.widgets = []

    def set_mesh(self, v, f, c=None):
        v = v.astype("float32", copy=False)
        f = f.astype("uint16", copy=False).ravel()
        
        if type(c) == type(None):
            c = np.ones_like(v)
            c[:, 1] = 0.874
            c[:, 2] = 0.0
            geometry = BufferGeometry(attributes=dict(
                position=BufferAttribute(v, normalized=False),
                index=BufferAttribute(f, normalized=False),
                color=BufferAttribute(c)
            ))
        else:
            c = c.astype("float32", copy=False)
            c = c[:, :3]
            geometry = BufferGeometry(attributes=dict(
                position=BufferAttribute(v, normalized=False),
                index=BufferAttribute(f, normalized=False),
                color=BufferAttribute(c),
            ))
        geometry.exec_three_obj_method('computeVertexNormals')
        self.mesh = Mesh(geometry=geometry, material=[MeshStandardMaterial(vertexColors='VertexColors', reflectivity=1.0, 
                                                            #side='FrontSide', 
                                                            roughness=0.5, metalness=0.25, flatShading=False,
                                                            polygonOffset=True, polygonOffsetFactor= 1,
                                                            polygonOffsetUnits=1)])
#MeshLambertMaterial(vertexColors='VertexColors')
        geon = WireframeGeometry(self.mesh.geometry) # WireframeGeometry
        mat = LineBasicMaterial(color="black", linewidth=0.6)
        wireframe = LineSegments( geon, mat )
        self.mesh.add(wireframe)

        self.scene = Scene(children=[self.mesh, self.cam, AmbientLight(intensity=0.5)], background="#4c4c80")
        self.renderer.scene = self.scene
        
    def add_widget(self, widget, callback):
        self.widgets.append(widget)
        widget.observe(callback, names='value')

    def launch(self):
        display(self.renderer)
        for w in self.widgets:
            display(w)


In [ ]:
from pynput import keyboard

def on_press(key):
    try:
        print('alphanumeric key {0} pressed'.format(
            key.char))
    except AttributeError:
        print('special key {0} pressed'.format(
            key))

def on_release(key):
    print('{0} released'.format(
        key))
    if key == keyboard.Key.esc:
        # Stop listener
        return False

# Collect events until released
with keyboard.Listener(
        on_press=on_press,
        on_release=on_release) as listener:
    listener.join()

In [ ]:
a